In [1]:
import torch
import pandas as pd

In [2]:
df = pd.read_excel('Flu-IC50.xlsx').dropna(how='all', axis='columns')
df.head()

,Title,IC50,SMILES
0,CHEMBL2206459,0.000015,[H]\N=C(N)\N[C@@H](C1)[C@@H](NC(=O)C)[C@@H](C=...
1,CHEMBL3818159,0.000016,O=C(O)C1=C[C@H](N)[C@@H](NC(=O)C)[C@@H](C1)COC...
2,CHEMBL1956716,0.000033,C=CC(=O)N[C@H]([C@H](C1)N)[C@@H](C=C1C([O-])=O...
3,CHEMBL1956715,0.000032,CC(=O)N[C@H]([C@H](C1)N)[C@@H](C=C1C([O-])=O)O...
4,CHEMBL4444029,0.000041,O=C(O)c1c(O)c(=O)cc([nH]1)-c(c2C)ccc(c2)-c3noc...


In [3]:
from transformers import AutoModelForCausalLM, AutoTokenizer

model_name = "ncfrey/ChemGPT-1.2B"
model = AutoModelForCausalLM.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [10]:
# tokenizer.pad_token = tokenizer.eos_token
tokenizer.add_special_tokens({'pad_token': '[PAD]'})
model.resize_token_embeddings(len(tokenizer))

Embedding(684, 2048)

In [12]:
smiles_data = list(df.head()['SMILES'])
inputs = tokenizer(smiles_data, padding=True, truncation=True, return_tensors="pt")

In [13]:
with torch.no_grad():
    outputs = model(**inputs)

In [15]:
outputs[0]

tensor([[[-7.1804, -9.7743, -2.2289,  ..., -9.0250, -8.9126, -9.0824],
         [-9.1802, -8.3181, -0.6276,  ..., -6.7926, -6.8580, -7.4653],
         [-2.8010, -1.1691,  1.8836,  ..., -1.5664, -1.2442, -1.5365],
         ...,
         [-4.7212, -5.0774, 13.2146,  ..., -3.5309, -3.4360, -5.0608],
         [-8.4461, -4.3602, -6.0253,  ..., -4.4558, -4.3697, -4.0440],
         [-8.5629, -4.5358, -6.7843,  ..., -4.6586, -4.6708, -4.3956]],

        [[-7.1804, -9.7743, -2.2289,  ..., -9.0250, -8.9126, -9.0824],
         [-9.2179, -9.2210, -1.9453,  ..., -7.9513, -7.9023, -8.5279],
         [-6.2971, -5.3869, -2.6357,  ..., -5.7580, -5.4007, -5.1342],
         ...,
         [-8.2413, -5.4834, -3.7402,  ..., -5.8144, -6.0123, -5.6413],
         [-8.4408, -5.0743, -5.0639,  ..., -5.5578, -5.7698, -5.3660],
         [-8.7549, -5.4699, -4.8865,  ..., -5.9070, -6.0358, -5.5829]],

        [[-7.1804, -9.7743, -2.2289,  ..., -9.0250, -8.9126, -9.0824],
         [-9.2179, -9.2210, -1.9453,  ..., -7